In [21]:
import pandas as pd
import numpy as np

In [89]:
df = pd.read_csv("train.csv")
df

,text,label
0,AUSTRALIAN PRODUCT INFORMATION,0
1,IMFINZI®,1
2,(durvalumab),1
3,1 \tNAME OF THE MEDICINE,0
4,Durvalumab,2
...,...,...
326,27 October 2021,2
327,Summary table of changes,2
328,Section changed Summary of new information 4.1...,2
329,© AstraZeneca 2021,2


In [90]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [91]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Reusing dataset imdb (C:\Users\vigne\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████| 3/3 [00:00<00:00, 230.64it/s]


In [92]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [93]:
# data_files = {"train": "train.csv", "test": "test.csv"}
data_files = {"train":"train.csv"}

In [94]:
df = load_dataset("csv", data_files=data_files)
df

Using custom data configuration default-0e6a807f287cdda0


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1006.07it/s]


Dataset csv downloaded and prepared to C:\Users\vigne\.cache\huggingface\datasets\csv\default-0e6a807f287cdda0\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 111.14it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 331
    })
})

In [99]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [96]:
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [97]:
df["train"][0]

{'text': 'AUSTRALIAN PRODUCT INFORMATION', 'label': 0}

In [100]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\vigne\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-b0a06afbb3821af6.arrow
100%|██████████| 25/25 [00:04<00:00,  5.74ba/s]
Loading cached processed dataset at C:\Users\vigne\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-56953b4a8b2a3288.arrow


In [102]:
tokenized_df = df.map(preprocess_function, batched = True)

100%|██████████| 1/1 [00:00<00:00, 51.72ba/s]


In [104]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [105]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [106]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [107]:
trainer.train()

c:\Users\vigne\anaconda3\envs\ptorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815
  0%|          | 0/7815 [00:00<?, ?it/s]The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 6.00 GiB total capacity; 4.43 GiB already allocated; 0 bytes free; 4.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF